In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
import socket
btns = base.btns_gpio


In [ ]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
#include <unistd.h>
#define PIN_BLU 5
#define PIN_GRN 6
#define PIN_RED 7
#define CODE_RED 0
#define CODE_BLU 1
#define CODE_GRN 2
#define CODE_TUR 3
#define CODE_PUR 4

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

void reset_gpio(void) {
    write_gpio(0,0);
    write_gpio(1,0);
    write_gpio(2,0);
    write_gpio(3,0);
    write_gpio(5,0);
    write_gpio(4,0);
    write_gpio(6,0);
    write_gpio(7,0);
}
void reset_led() {
    write_gpio(5,0);
    write_gpio(6,0);
    write_gpio(7,0);
}


//Color function
void write_color(int color_code, int on_off) {
    if (color_code == 0) { //red
        write_gpio(PIN_BLU, 0);
        write_gpio(PIN_GRN, 0);
        write_gpio(PIN_RED, on_off);
    }
    else if (color_code == 1) { //blue
        write_gpio(PIN_BLU, on_off);
        write_gpio(PIN_GRN, 0);
        write_gpio(PIN_RED, 0);
    }
    else if (color_code == 2) { //green
        write_gpio(PIN_BLU, 0);
        write_gpio(PIN_GRN, on_off);
        write_gpio(PIN_RED, 0);
    }
    else if (color_code == 3) { //turqoise
        write_gpio(PIN_BLU, on_off);
        write_gpio(PIN_GRN, on_off);
        write_gpio(PIN_RED, 0);
    }
    else if (color_code == 4) { //purple
        write_gpio(PIN_BLU, on_off);
        write_gpio(PIN_GRN, 0);
        write_gpio(PIN_RED, on_off);
    }
}




In [ ]:
%%microblaze base.PMODA
#include "i2c.h"
#define PMOD_PIN___SCL 3
#define PMOD_PIN___SDA 2

unsigned int read_temp(void) {
    i2c device = i2c_open(PMOD_PIN___SDA, PMOD_PIN___SCL);

    unsigned char buf[2];
    unsigned int upper, lower;

    //first, write 0b0101 (code for temperature register) to pointer register:
    buf[0] = 0x05;
    i2c_write(device, 0x18, buf, 1); //0x30

    //lastly, read temperature register:
    i2c_read(device, 0x18, buf, 2);

    //return temperature unsigned int:
    //i2c_close(device);
    upper = (unsigned int) buf[0];
    upper &= 0x0F;
    lower = (unsigned int) buf[1];
    lower &= 0xFF;
    return((upper << 8) | lower);
}

In [ ]:
#reset_gpio()
duty_cycle = 50
color = 3
def RGB_LED_PWM():
    global duty_cycle
    global color

    while True:
        if duty_cycle > 100 or duty_cycle < 0:
            print("Invalid duty cycle entered. Enter DC between 0 and 100 percent (integer) inclusive.")
        elif duty_cycle == 0:
            write_color(color, 0) #needed due to finite time it takes to run the time.sleep() method, despite the argument being 0.0sec
        elif duty_cycle == 100:
            write_color(color, 1) #From what I saw, this if clause is not really needed as the OFF blip was imperceptible when running the else clause below with duty_cycle==100
        else:
            period_sec = 1.0/10.0
            sleep_time_ON = (duty_cycle/100.0)*period_sec
            #print(str(sleep_time_ON) + " seconds") #debug statement
            sleep_time_OFF = period_sec - sleep_time_ON
            #print(str(sleep_time_OFF) + " seconds") #debug statement
            write_color(color, 1)
            time.sleep(sleep_time_ON)
            write_color(color, 0)
            time.sleep(sleep_time_OFF)


def convertTempCodeToFloat(code):
    #incoming code is (upper << 8) | lower)
    upper = (code & 0x0F00) >> 8
    lower = code & 0x00FF
    return ((upper * (2**4)) + (lower * (1.0/(2**4))))

In [ ]:
def client():
    global duty_cycle,color
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    print("Client side running")
# 1: Connect the socket (sock) to the <SERVER-IP> and choosen port <LISTENING-PORT>
    sock.connect(('192.168.2.99', 1251))
    while True:
        temp = convertTempCodeToFloat(read_temp())
        time.sleep(.5)
        print('temp: ',temp)
        sock.send(str(temp).encode())
        duty_cycle = int(sock.recv(1024).decode())
        print('duty cycle: ', duty_cycle)
        color = int(sock.recv(1024).decode())
        print('color:', color )

# 3: Close the socket
    sock.close()

In [ ]:
from multiprocessing import Process
from threading import Thread, Lock
if __name__ == '__main__':
    fork = Lock()
    p = Thread(target=RGB_LED_PWM)
    f = Thread(target=client)
    p.start()
    f.start()
    f.join()
    p.join()

Client side running
temp:  22.375
duty cycle:  50
color: 3
temp:  22.375
duty cycle:  50
color: 3
temp:  22.375
duty cycle:  50
color: 3
temp:  22.375
duty cycle:  50
color: 4
temp:  22.375
duty cycle:  50
color: 4
temp:  22.4375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  60
color: 0
temp:  22.375
duty cycle:  70
color: 0
temp:  22.375
duty cycle:  70
color: 0
temp:  22.375
duty cycle:  60
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.4375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.375
duty cycle:  50
color: 0
temp:  22.9375
duty cycle:  50
color: 0
temp:  23.125
duty cycle:  50
color: 0
te

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.8/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag